In [3]:
!pip install docplex
!pip install cplex

   ---------------------------------------- 0.0/26.3 MB ? eta -:--:--
   - -------------------------------------- 1.0/26.3 MB 16.7 MB/s eta 0:00:02
   ---------- ----------------------------- 7.1/26.3 MB 18.2 MB/s eta 0:00:02
   ------------ --------------------------- 8.4/26.3 MB 16.8 MB/s eta 0:00:02
   ----------------- ---------------------- 11.5/26.3 MB 14.4 MB/s eta 0:00:02
   ----------------------- ---------------- 15.7/26.3 MB 15.7 MB/s eta 0:00:01
   --------------------------- ------------ 17.8/26.3 MB 15.8 MB/s eta 0:00:01
   --------------------------- ------------ 17.8/26.3 MB 15.8 MB/s eta 0:00:01
   ----------------------------- ---------- 19.1/26.3 MB 11.5 MB/s eta 0:00:01
   --------------------------------- ------ 22.0/26.3 MB 11.9 MB/s eta 0:00:01
   ----------------------------------- ---- 23.1/26.3 MB 11.1 MB/s eta 0:00:01
   ------------------------------------- -- 24.4/26.3 MB 10.5 MB/s eta 0:00:01
   ---------------------------------------- 26.3/26.3 MB 10.6 MB

In [1]:
from docplex.mp.model import Model

# 데이터 정의
risk_free = [[0.03], [0.015], [0.01]]  # 현Gold 수익률 (1years, 3years, 5years)
r = [
    [0.262, 0.03, 0.012, 0.127],  # Stocks, Long-term Bonds, Short-term Bonds, Gold (1years)
    [0.1, 0.015, -0.002, 0.023],  # 3years
    [0.156, 0.02, 0.012, 0.095]   # 5years
]
σ = [
    [0.147, 0.05, 0.019, 0.151],  # 1years
    [0.176, 0.06, 0.022, 0.142],  # 3years
    [0.185, 0.07, 0.019, 0.145]   # 5years
]
mdd = [
    [0.083, 0.03, 0.054, 0.072],  # 1years
    [0.239, 0.04, 0.054, 0.159],  # 3years
    [0.239, 0.05, 0.054, 0.181]   # 5years
]
year_mapping = {1: 0, 3: 1, 5: 2}

# 세율 정의
income_tax = 0.154
corp_tax = 0.19
div_tax = 0.15
transaction_tax = 0.0023

# 1단계: 전체 최적화 (현Gold 포함)
def optimize_full_portfolio(y1_val, y2_val, y3_val, year):
    year_idx = year_mapping[year]
    m = Model(name=f"Full Portfolio Optimization - {year} Years")
    
    # 변수 정의
    x = [m.continuous_var(name=f"x{i+1}", lb=0) for i in range(5)]  # Stocks, Long-term Bonds, Short-term Bonds, Gold, 현Gold
    
    # 자산 비중 합 제약
    m.add_constraint(m.sum(x) == 1, "Asset_Allocation_Sum")
    
    # 위험 제약
    portfolio_volatility = m.sum(σ[year_idx][i] * x[i] for i in range(4))
    portfolio_mdd = m.sum(mdd[year_idx][i] * x[i] for i in range(4))
    
    if y1_val == 1:  # Conservative
        m.add_constraint(portfolio_volatility <= 0.103, "Volatility_Stable")
        m.add_constraint(portfolio_mdd <= 0.21, "MaxDrawdown_Stable")
    elif y2_val == 1:  # Neutral
        m.add_constraint(portfolio_volatility <= 0.134, "Volatility_Neutral")
        m.add_constraint(portfolio_mdd <= 0.23, "MaxDrawdown_Neutral")
    elif y3_val == 1:  # Aggressive
        m.add_constraint(portfolio_volatility <= 0.144, "Volatility_Aggressive")
        m.add_constraint(portfolio_mdd <= 0.29, "MaxDrawdown_Aggressive")
    
    # 목표 함수: 세후 수익률 최대화
    adj_r = [
        r[year_idx][0] * (1 - div_tax) - transaction_tax,  # Stocks
        r[year_idx][1] * (1 - income_tax),                # Long-term Bonds
        r[year_idx][2] * (1 - income_tax),                # Short-term Bonds
        r[year_idx][3] * (1 - corp_tax),                  # Gold
        risk_free[year_idx][0] * (1 - income_tax)         # 현Gold
    ]
    m.set_objective("max", m.sum(adj_r[i] * x[i] for i in range(4)))
    
    solution = m.solve()
    if solution:
        cash_ratio = solution[x[4]]
        portfolio_volatility_val = portfolio_volatility.solution_value
        portfolio_mdd_val = portfolio_mdd.solution_value
        print(f"[Step 1: Cash/Gold Allocation & Risk Hedging] Period: {year}years, Investor Type: Conservative={y1_val}, Neutral={y2_val}, Aggressive={y3_val}")
        print(f"  - Cash/Gold allocation is set to {cash_ratio:.2%} of the portfolio.")
        print("\n")
#         print(f"  - Volatility(Volatility): {portfolio_volatility_val:.2%}")
#         print(f"  - Maximum Drawdown (MDD): {portfolio_mdd_val:.2%}")
#         print("-" * 50)
        return cash_ratio
    else:
        print(f"[Step 1: Cash/Gold Optimization Failed] Period: {year}years, Investor Type: Conservative={y1_val}, Neutral={y2_val}, Aggressive={y3_val}")
        print("-" * 50)
        return None

# 2단계: 나머지 자산 최적화
def optimize_remaining_portfolio(cash_ratio, y1_val, y2_val, y3_val, year):
    year_idx = year_mapping[year]
    m = Model(name=f"Remaining Portfolio Optimization - {year} Years")
    
    # 변수 정의: Stocks, Long-term Bonds, Short-term Bonds, Gold
    x = [m.continuous_var(name=f"x{i+1}", lb=0) for i in range(4)]
    
    # 자산 비중 합 제약: 나머지 비중 합은 1 - cash_ratio
    remaining_ratio = 1 - cash_ratio
    m.add_constraint(m.sum(x) == remaining_ratio, "Remaining_Allocation_Sum")
    
    # 위험 제약
    portfolio_volatility = m.sum(σ[year_idx][i] * x[i] for i in range(4))
    portfolio_mdd = m.sum(mdd[year_idx][i] * x[i] for i in range(4))
    
    if y1_val == 1:  # Conservative
        m.add_constraint(portfolio_volatility <= 0.103 * remaining_ratio, "Volatility_Stable")
        m.add_constraint(portfolio_mdd <= 0.21 * remaining_ratio, "MaxDrawdown_Stable")
    elif y2_val == 1:  # Neutral
        m.add_constraint(portfolio_volatility <= 0.134 * remaining_ratio, "Volatility_Neutral")
        m.add_constraint(portfolio_mdd <= 0.23 * remaining_ratio, "MaxDrawdown_Neutral")
    elif y3_val == 1:  # Aggressive
        m.add_constraint(portfolio_volatility <= 0.144 * remaining_ratio, "Volatility_Aggressive")
        m.add_constraint(portfolio_mdd <= 0.29 * remaining_ratio, "MaxDrawdown_Aggressive")
    
    # 목표 함수: 세후 수익률 최대화
    adj_r = [
        r[year_idx][0] * (1 - div_tax) - transaction_tax,  # Stocks
        r[year_idx][1] * (1 - income_tax),                # Long-term Bonds
        r[year_idx][2] * (1 - income_tax),                # Short-term Bonds
        r[year_idx][3] * (1 - corp_tax)                   # Gold
    ]
    m.set_objective("max", m.sum(adj_r[i] * x[i] for i in range(4)))
    
    solution = m.solve()
    if solution:
        portfolio_volatility_val = portfolio_volatility.solution_value
        portfolio_mdd_val = portfolio_mdd.solution_value
        print(f"[Step 2: Asset Allocation after Risk Hedging]")
        for i, asset_name in enumerate(["Stocks", "Long-term Bonds", "Short-term Bonds", "Gold"]):
            print(f"  - {asset_name}: {solution[x[i]]:.2%}")
        print(f"  - Maximum Expected Return (Objective): {solution.objective_value:.2%}")
        print(f"  - Maximum Allowed Volatility: {portfolio_volatility_val:.2%}")
        print(f"  - Maximum Drawdown (MDD): {portfolio_mdd_val:.2%}")
        print("-" * 50)
        print("\n")
    else:
        print(f"[Step 2: Remaining Asset Optimization Failed] Period: {year}years, Investor Type: Conservative={y1_val}, Neutral={y2_val}, Aggressive={y3_val}")
        print("-" * 50)

# 실행
for year in [1, 3, 5]:
    for y1_val in [1, 0]:
        for y2_val in [1, 0]:
            for y3_val in [1, 0]:
                if y1_val + y2_val + y3_val == 1:
                    cash_ratio = optimize_full_portfolio(y1_val, y2_val, y3_val, year)
                    if cash_ratio is not None:
                        optimize_remaining_portfolio(cash_ratio, y1_val, y2_val, y3_val, year)


[Step 1: Cash/Gold Allocation & Risk Hedging] Period: 1years, Investor Type: Conservative=1, Neutral=0, Aggressive=0
  - Cash/Gold allocation is set to 29.93% of the portfolio.


[Step 2: Asset Allocation after Risk Hedging]
  - Stocks: 45.98%
  - Long-term Bonds: 0.00%
  - Short-term Bonds: 24.09%
  - Gold: 0.00%
  - Maximum Expected Return (Objective): 10.38%
  - Maximum Allowed Volatility: 7.22%
  - Maximum Drawdown (MDD): 5.12%
--------------------------------------------------


[Step 1: Cash/Gold Allocation & Risk Hedging] Period: 1years, Investor Type: Conservative=0, Neutral=1, Aggressive=0
  - Cash/Gold allocation is set to 8.84% of the portfolio.


[Step 2: Asset Allocation after Risk Hedging]
  - Stocks: 81.90%
  - Long-term Bonds: 0.00%
  - Short-term Bonds: 9.26%
  - Gold: 0.00%
  - Maximum Expected Return (Objective): 18.14%
  - Maximum Allowed Volatility: 12.21%
  - Maximum Drawdown (MDD): 7.30%
--------------------------------------------------


[Step 1: Cash/Gold Allo

In [9]:
from docplex.mp.model import Model

# 데이터 정의
risk_free = [[0.051], [0.023], [0.019]]  # 현Gold 수익률 (1years, 3years, 5years)
r = [
    [0.262, 0.03, 0.012, 0.127],  # Stocks, Long-term Bonds, Short-term Bonds, Gold (1years)
    [0.1, 0.015, -0.002, 0.023],  # 3years
    [0.156, 0.02, 0.012, 0.095]   # 5years
]
σ = [
    [0.147, 0.05, 0.019, 0.151],  # 1years
    [0.176, 0.06, 0.022, 0.142],  # 3years
    [0.185, 0.07, 0.019, 0.145]   # 5years
]
mdd = [
    [0.083, 0.03, 0.054, 0.072],  # 1years
    [0.239, 0.04, 0.054, 0.159],  # 3years
    [0.239, 0.05, 0.054, 0.181]   # 5years
]
year_mapping = {1: 0, 3: 1, 5: 2}

# 세율 정의
income_tax = 0.154
corp_tax = 0.19
div_tax = 0.15
transaction_tax = 0.0023

# 1단계: 전체 최적화 (현Gold 포함)
def optimize_full_portfolio(y1_val, y2_val, y3_val, year):
    year_idx = year_mapping[year]
    m = Model(name=f"Full Portfolio Optimization - {year} Years")
    
    # 변수 정의
    x = [m.continuous_var(name=f"x{i+1}", lb=0) for i in range(5)]  # Stocks, Long-term Bonds, Short-term Bonds, Gold, 현Gold
    
    # 자산 비중 합 제약
    m.add_constraint(m.sum(x) == 1, "Asset_Allocation_Sum")
    
    # 위험 제약
    portfolio_volatility = m.sum(σ[year_idx][i] * x[i] for i in range(4))
    portfolio_mdd = m.sum(mdd[year_idx][i] * x[i] for i in range(4))
    
    if y1_val == 1:  # Conservative
        m.add_constraint(portfolio_volatility <= 0.103, "Volatility_Stable")
        m.add_constraint(portfolio_mdd <= 0.21, "MaxDrawdown_Stable")
    elif y2_val == 1:  # Neutral
        m.add_constraint(portfolio_volatility <= 0.134, "Volatility_Neutral")
        m.add_constraint(portfolio_mdd <= 0.23, "MaxDrawdown_Neutral")
    elif y3_val == 1:  # Aggressive
        m.add_constraint(portfolio_volatility <= 0.144, "Volatility_Aggressive")
        m.add_constraint(portfolio_mdd <= 0.29, "MaxDrawdown_Aggressive")
    
    # 목표 함수: 세후 수익률 최대화
    adj_r = [
        r[year_idx][0] * (1 - div_tax) - transaction_tax,  # Stocks
        r[year_idx][1] * (1 - income_tax),                # Long-term Bonds
        r[year_idx][2] * (1 - income_tax),                # Short-term Bonds
        r[year_idx][3] * (1 - corp_tax),                  # Gold
        risk_free[year_idx][0] * (1 - income_tax)         # 현Gold
    ]
    m.set_objective("max", m.sum(adj_r[i] * x[i] for i in range(4)))
    
    solution = m.solve()
    if solution:
        cash_ratio = solution[x[4]]
#         portfolio_volatility_val = portfolio_volatility.solution_value
#         portfolio_mdd_val = portfolio_mdd.solution_value
        print(f"[Step 1: Cash/Gold Allocation & Risk Hedging] Period: {year}years, Investor Type: Conservative={y1_val}, Neutral={y2_val}, Aggressive={y3_val}")
        print(f"  - Cash/Gold allocation is set to {cash_ratio:.2%} of the portfolio.")
        print("\n")
#         print(f"  - Volatility(Volatility): {portfolio_volatility_val:.2%}")
#         print(f"  - Maximum Drawdown (MDD): {portfolio_mdd_val:.2%}")
#         print("-" * 50)
        return cash_ratio
    else:
        print(f"[Step 1: Cash/Gold Optimization Failed] Period: {year}years, Investor Type: Conservative={y1_val}, Neutral={y2_val}, Aggressive={y3_val}")
        print("-" * 50)
        return None

# 2단계: 나머지 자산 최적화
def optimize_remaining_portfolio(cash_ratio, y1_val, y2_val, y3_val, year):
    year_idx = year_mapping[year]
    m = Model(name=f"Remaining Portfolio Optimization - {year} Years")
    
    # 변수 정의: Stocks, Long-term Bonds, Short-term Bonds, Gold
    x = [m.continuous_var(name=f"x{i+1}", lb=0) for i in range(4)]
    
    # 자산 비중 합 제약: 나머지 비중 합은 1 - cash_ratio
    remaining_ratio = 1 - cash_ratio
    m.add_constraint(m.sum(x) == remaining_ratio, "Remaining_Allocation_Sum")
    
    # 위험 제약
    portfolio_volatility = m.sum(σ[year_idx][i] * x[i] for i in range(4))
    portfolio_mdd = m.sum(mdd[year_idx][i] * x[i] for i in range(4))
    
    if y1_val == 1:  # Conservative
        m.add_constraint(portfolio_volatility <= 0.103 * remaining_ratio, "Volatility_Stable")
        m.add_constraint(portfolio_mdd <= 0.21 * remaining_ratio, "MaxDrawdown_Stable")
    elif y2_val == 1:  # Neutral
        m.add_constraint(portfolio_volatility <= 0.134 * remaining_ratio, "Volatility_Neutral")
        m.add_constraint(portfolio_mdd <= 0.23 * remaining_ratio, "MaxDrawdown_Neutral")
    elif y3_val == 1:  # Aggressive
        m.add_constraint(portfolio_volatility <= 0.144 * remaining_ratio, "Volatility_Aggressive")
        m.add_constraint(portfolio_mdd <= 0.29 * remaining_ratio, "MaxDrawdown_Aggressive")
    
    # 목표 함수: 세후 수익률 최대화
    adj_r = [
        r[year_idx][0] * (1 - div_tax) - transaction_tax,  # Stocks
        r[year_idx][1] * (1 - income_tax),                # Long-term Bonds
        r[year_idx][2] * (1 - income_tax),                # Short-term Bonds
        r[year_idx][3] * (1 - corp_tax)                   # Gold
    ]
    m.set_objective("max", m.sum(adj_r[i] * x[i] for i in range(4)))
    
    solution = m.solve()
    if solution:
        portfolio_volatility_val = portfolio_volatility.solution_value
        portfolio_mdd_val = portfolio_mdd.solution_value
        print(f"[Step 2: Asset Allocation after Risk Hedging]")
        for i, asset_name in enumerate(["Stocks", "Long-term Bonds", "Short-term Bonds", "Gold"]):
            print(f"  - {asset_name}: {solution[x[i]]:.2%}")
        print(f"  - Maximum Expected Return (Objective): {solution.objective_value:.2%}")
        print(f"  - Maximum Allowed Volatility: {portfolio_volatility_val:.2%}")
        print(f"  - Maximum Drawdown (MDD): {portfolio_mdd_val:.2%}")
        print("-" * 50)
        print("\n")
    else:
        print(f"[Step 2: Remaining Asset Optimization Failed] Period: {year}years, Investor Type: Conservative={y1_val}, Neutral={y2_val}, Aggressive={y3_val}")
        print("-" * 50)

# 실행
for year in [1, 3, 5]:
    for y1_val in [1, 0]:
        for y2_val in [1, 0]:
            for y3_val in [1, 0]:
                if y1_val + y2_val + y3_val == 1:
                    cash_ratio = optimize_full_portfolio(y1_val, y2_val, y3_val, year)
                    if cash_ratio is not None:
                        optimize_remaining_portfolio(cash_ratio, y1_val, y2_val, y3_val, year)


[Step 1: Cash/Gold Allocation & Risk Hedging] Period: 1years, Investor Type: Conservative=1, Neutral=0, Aggressive=0
  - Cash/Gold allocation is set to 29.93% of the portfolio.


[Step 2: Asset Allocation after Risk Hedging]
  - Stocks: 45.98%
  - Long-term Bonds: 0.00%
  - Short-term Bonds: 24.09%
  - Gold: 0.00%
  - Maximum Expected Return (Objective): 10.38%
  - Maximum Allowed Volatility: 7.22%
  - Maximum Drawdown (MDD): 5.12%
--------------------------------------------------


[Step 1: Cash/Gold Allocation & Risk Hedging] Period: 1years, Investor Type: Conservative=0, Neutral=1, Aggressive=0
  - Cash/Gold allocation is set to 8.84% of the portfolio.


[Step 2: Asset Allocation after Risk Hedging]
  - Stocks: 81.90%
  - Long-term Bonds: 0.00%
  - Short-term Bonds: 9.26%
  - Gold: 0.00%
  - Maximum Expected Return (Objective): 18.14%
  - Maximum Allowed Volatility: 12.21%
  - Maximum Drawdown (MDD): 7.30%
--------------------------------------------------


[Step 1: Cash/Gold Allo

In [8]:
from docplex.mp.model import Model

# 데이터 정의
risk_free_r = [[0.051], [0.023], [0.019]]  # 현Gold 수익률 (1years, 3years, 5years)
r = [
    [0.262, 0.03, 0.012, 0.127],  # Stocks, Long-term Bonds, Short-term Bonds, Gold (1years)
    [0.1, 0.015, -0.002, 0.023],  # 3years
    [0.156, 0.02, 0.012, 0.095]   # 5years
]
σ = [
    [0.147, 0.05, 0.019, 0.151],  # 1years
    [0.176, 0.06, 0.022, 0.142],  # 3years
    [0.185, 0.07, 0.019, 0.145]   # 5years
]
mdd = [
    [0.083, 0.03, 0.054, 0.072],  # 1years
    [0.239, 0.04, 0.054, 0.159],  # 3years
    [0.239, 0.05, 0.054, 0.181]   # 5years
]
year_mapping = {1: 0, 3: 1, 5: 2}

# 세율 정의
income_tax = 0.154
corp_tax = 0.19
div_tax = 0.15
transaction_tax = 0.0023

# 1단계: 전체 최적화 (현Gold 포함)
def optimize_full_portfolio(y1_val, y2_val, y3_val, year):
    year_idx = year_mapping[year]
    m = Model(name=f"Full Portfolio Optimization - {year} Years")
    
    # 변수 정의
    x = [m.continuous_var(name=f"x{i+1}", lb=0) for i in range(4)]  # Stocks, Long-term Bonds, Short-term Bonds, Gold
    c = m.continuous_var(name="cash", lb=0)  # 현Gold

    # 자산 비중 합 제약
    m.add_constraint(m.sum(x) + c == 1, "Asset_Allocation_Sum")
    
    # 위험 제약
    portfolio_volatility = m.sum(σ[year_idx][i] * x[i] for i in range(4))
    portfolio_mdd = m.sum(mdd[year_idx][i] * x[i] for i in range(4))
    
    if y1_val == 1:  # Conservative
        m.add_constraint(portfolio_volatility <= 0.103, "Volatility_Stable")
        m.add_constraint(portfolio_mdd <= 0.21, "MaxDrawdown_Stable")
    elif y2_val == 1:  # Neutral
        m.add_constraint(portfolio_volatility <= 0.134, "Volatility_Neutral")
        m.add_constraint(portfolio_mdd <= 0.23, "MaxDrawdown_Neutral")
    elif y3_val == 1:  # Aggressive
        m.add_constraint(portfolio_volatility <= 0.144, "Volatility_Aggressive")
        m.add_constraint(portfolio_mdd <= 0.29, "MaxDrawdown_Aggressive")
    
    # 목표 함수: 세후 수익률 최대화
    adj_r = [
        r[year_idx][0] * (1 - div_tax) - transaction_tax,  # Stocks
        r[year_idx][1] * (1 - income_tax),                # Long-term Bonds
        r[year_idx][2] * (1 - income_tax),                # Short-term Bonds
        r[year_idx][3] * (1 - corp_tax),                  # Gold
    ]
    adj_risk_free_r= risk_free_r[year_idx][0] * (1 - income_tax)         # 현Gold
    
    m.set_objective("max", m.sum(adj_r[i] * x[i] for i in range(4)) + adj_risk_free_r * c)
    cash_ratio=adj_risk_free_r * c
    
    solution = m.solve()
    if solution:
        cash_ratio = solution[c]
        portfolio_volatility_val = portfolio_volatility.solution_value
        portfolio_mdd_val = portfolio_mdd.solution_value
        print(f"[Step 1: Cash/Gold Allocation & Risk Hedging] Period: {year}years, Investor Type: Conservative={y1_val}, Neutral={y2_val}, Aggressive={y3_val}")
        print(f"  - Cash/Gold allocation is set to {cash_ratio:.2%} of the portfolio.\n")
        return cash_ratio
    else:
        print(f"[Step 1: Cash/Gold Optimization Failed] Period: {year}years, Investor Type: Conservative={y1_val}, Neutral={y2_val}, Aggressive={y3_val}")
        return None

# 2단계: 나머지 자산 최적화
def optimize_remaining_portfolio(cash_ratio, y1_val, y2_val, y3_val, year):
    year_idx = year_mapping[year]
    m = Model(name=f"Remaining Portfolio Optimization - {year} Years")
    
    # 변수 정의: Stocks, Long-term Bonds, Short-term Bonds, Gold
    x = [m.continuous_var(name=f"x{i+1}", lb=0) for i in range(4)]
    
    # 자산 비중 합 제약: 나머지 비중 합은 1 - cash_ratio
    remaining_ratio = 1 - cash_ratio
    m.add_constraint(m.sum(x) == remaining_ratio, "Remaining_Allocation_Sum")
    
#     # 무위험 자산 제약
#     cash_volatility = m.sum(σ[year_idx][i] * x[i] for i in range(4))
#     cash_mdd = m.sum(mdd[year_idx][i] * x[i] for i in range(4))
    
#     if y1_val == 1:  # Conservative
#         m.add_constraint(cash_volatility <= 0.00000001 * cash_ratio, "Volatility_Stable")
#         m.add_constraint(cash_mdd <= 0.0000001 * cash_ratio, "MaxDrawdown_Stable")
#     elif y2_val == 1:  # Neutral
#         m.add_constraint(cash_volatility <= 0.00000001 * cash_ratio, "Volatility_Neutral")
#         m.add_constraint(cash_mdd <= 0.00000001 * cash_ratio, "MaxDrawdown_Neutral")
#     elif y3_val == 1:  # Aggressive
#         m.add_constraint(portfol_volatility <= 0.000000002 * cash_ratio, "Volatility_Aggressive")
#         m.add_constraint(portfolio_mdd <= 0.00000003 * cash_ratio, "MaxDrawdown_Aggressive")
    
    # 위험 제약
    portfolio_volatility = m.sum(σ[year_idx][i] * x[i] for i in range(4))
    portfolio_mdd = m.sum(mdd[year_idx][i] * x[i] for i in range(4))
    
    if y1_val == 1:  # Conservative
        m.add_constraint(portfolio_volatility <= 0.103 * remaining_ratio, "Volatility_Stable")
        m.add_constraint(portfolio_mdd <= 0.21 * remaining_ratio, "MaxDrawdown_Stable")
    elif y2_val == 1:  # Neutral
        m.add_constraint(portfolio_volatility <= 0.134 * remaining_ratio, "Volatility_Neutral")
        m.add_constraint(portfolio_mdd <= 0.23 * remaining_ratio, "MaxDrawdown_Neutral")
    elif y3_val == 1:  # Aggressive
        m.add_constraint(portfolio_volatility <= 0.144 * remaining_ratio, "Volatility_Aggressive")
        m.add_constraint(portfolio_mdd <= 0.29 * remaining_ratio, "MaxDrawdown_Aggressive")
    
    # 목표 함수: 세후 수익률 최대화
    adj_r = [
        r[year_idx][0] * (1 - div_tax) - transaction_tax,  # Stocks
        r[year_idx][1] * (1 - income_tax),                # Long-term Bonds
        r[year_idx][2] * (1 - income_tax),                # Short-term Bonds
        r[year_idx][3] * (1 - corp_tax)                   # Gold
    ]
    m.set_objective("max", m.sum(adj_r[i] * x[i] for i in range(4)))
    
    solution = m.solve()
    if solution:
        portfolio_volatility_val = portfolio_volatility.solution_value
        portfolio_mdd_val = portfolio_mdd.solution_value
        print(f"[Step 2: Asset Allocation after Risk Hedging]")
        for i, asset_name in enumerate(["Stocks", "Long-term Bonds", "Short-term Bonds", "Gold"]):
            print(f"  - {asset_name}: {solution[x[i]]:.2%}")
        print(f"  - Maximum Expected Return (Objective): {solution.objective_value:.2%}")
        print(f"  - Maximum Allowed Volatility: {portfolio_volatility_val:.2%}")
        print(f"  - Maximum Drawdown (MDD): {portfolio_mdd_val:.2%}")
        print("-" * 50)
    else:
        print(f"[Step 2: Remaining Asset Optimization Failed] Period: {year}years, Investor Type: Conservative={y1_val}, Neutral={y2_val}, Aggressive={y3_val}")
        print("-" * 50)

# 실행
for year in [1, 3, 5]:
    for y1_val in [1, 0]:
        for y2_val in [1, 0]:
            for y3_val in [1, 0]:
                if y1_val + y2_val + y3_val == 1:
                    cash_ratio = optimize_full_portfolio(y1_val, y2_val, y3_val, year)
                    if cash_ratio is not None:
                        optimize_remaining_portfolio(cash_ratio, y1_val, y2_val, y3_val, year)



[Step 1: Cash/Gold Allocation & Risk Hedging] Period: 1years, Investor Type: Conservative=1, Neutral=0, Aggressive=0
  - Cash/Gold allocation is set to 29.93% of the portfolio.

[Step 2: Asset Allocation after Risk Hedging]
  - Stocks: 45.98%
  - Long-term Bonds: 0.00%
  - Short-term Bonds: 24.09%
  - Gold: 0.00%
  - Maximum Expected Return (Objective): 10.38%
  - Maximum Allowed Volatility: 7.22%
  - Maximum Drawdown (MDD): 5.12%
--------------------------------------------------
[Step 1: Cash/Gold Allocation & Risk Hedging] Period: 1years, Investor Type: Conservative=0, Neutral=1, Aggressive=0
  - Cash/Gold allocation is set to 8.84% of the portfolio.

[Step 2: Asset Allocation after Risk Hedging]
  - Stocks: 81.90%
  - Long-term Bonds: 0.00%
  - Short-term Bonds: 9.26%
  - Gold: 0.00%
  - Maximum Expected Return (Objective): 18.14%
  - Maximum Allowed Volatility: 12.21%
  - Maximum Drawdown (MDD): 7.30%
--------------------------------------------------
[Step 1: Cash/Gold Allocation

In [4]:
from docplex.mp.model import Model

# 데이터 정의
risk_free_r = [[0.051], [0.023], [0.019]]  # 현Gold 수익률 (1years, 3years, 5years)
r = [
    [0.262, 0.028, 0.042, 0.127],  # Stocks, Long-term Bonds, Short-term Bonds, Gold (1years)
    [0.1, -0.123, -0.002, 0.023],  # 3years
    [0.156, -0.019,0.012, 0.095]   # 5years
]
σ = [
    [0.147, 0.208, 0.025, 0.151],  # 1years
    [0.176, 0.169, 0.022, 0.142],  # 3years
    [0.185, 0.163, 0.019, 0.145]   # 5years
]
mdd = [
    [0.083, 0.201, 0.009, 0.072],  # 1years
    [0.239, 0.436, 0.054, 0.159],  # 3years
    [0.239, 0.476, 0.054, 0.181]   # 5years
]
year_mapping = {1: 0, 3: 1, 5: 2}

# 세율 정의
income_tax = 0.154
corp_tax = 0.19
div_tax = 0.15
transaction_tax = 0.0023

# 1단계: 전체 최적화 (현Gold 포함)
def optimize_full_portfolio(y1_val, y2_val, y3_val, year):
    year_idx = year_mapping[year]
    m = Model(name=f"Full Portfolio Optimization - {year} Years")
    
    # 변수 정의
    x = [m.continuous_var(name=f"x{i+1}", lb=0) for i in range(4)]  # Stocks, Long-term Bonds, Short-term Bonds, Gold
    c = m.continuous_var(name="cash", lb=0)  # 현Gold

    # 자산 비중 합 제약
    m.add_constraint(m.sum(x) + c == 1, "Asset_Allocation_Sum")
    
    # 위험 제약
    portfolio_volatility = m.sum(σ[year_idx][i] * x[i] for i in range(4))
    portfolio_mdd = m.sum(mdd[year_idx][i] * x[i] for i in range(4))
    
    if y1_val == 1:  # Conservative
        m.add_constraint(portfolio_volatility <= 0.103, "Volatility_Stable")
        m.add_constraint(portfolio_mdd <= 0.21, "MaxDrawdown_Stable")
    elif y2_val == 1:  # Neutral
        m.add_constraint(portfolio_volatility <= 0.134, "Volatility_Neutral")
        m.add_constraint(portfolio_mdd <= 0.23, "MaxDrawdown_Neutral")
    elif y3_val == 1:  # Aggressive
        m.add_constraint(portfolio_volatility <= 0.144, "Volatility_Aggressive")
        m.add_constraint(portfolio_mdd <= 0.29, "MaxDrawdown_Aggressive")
    
    # 목표 함수: 세후 수익률 최대화
    adj_r = [
        r[year_idx][0] * (1 - div_tax) - transaction_tax,  # Stocks
        r[year_idx][1] * (1 - income_tax),                # Long-term Bonds
        r[year_idx][2] * (1 - income_tax),                # Short-term Bonds
        r[year_idx][3] * (1 - corp_tax),                  # Gold
    ]
    adj_risk_free_r= risk_free_r[year_idx][0] * (1 - income_tax)         # 현Gold
    
    m.set_objective("max", m.sum(adj_r[i] * x[i] for i in range(4)) + adj_risk_free_r * c)
    cash_ratio=adj_risk_free_r * c
    
    solution = m.solve()
    if solution:
        cash_ratio = solution[c]
        portfolio_volatility_val = portfolio_volatility.solution_value
        portfolio_mdd_val = portfolio_mdd.solution_value
        print(f"[Step 1: Cash/Gold Allocation & Risk Hedging] Period: {year}years, Investor Type: Conservative={y1_val}, Neutral={y2_val}, Aggressive={y3_val}")
        print(f"  - Cash/Gold allocation is set to {cash_ratio:.2%} of the portfolio.\n")
        return cash_ratio
    else:
        print(f"[Step 1: Cash/Gold Optimization Failed] Period: {year}years, Investor Type: Conservative={y1_val}, Neutral={y2_val}, Aggressive={y3_val}")
        return None

# 2단계: 나머지 자산 최적화
def optimize_remaining_portfolio(cash_ratio, y1_val, y2_val, y3_val, year):
    year_idx = year_mapping[year]
    m = Model(name=f"Remaining Portfolio Optimization - {year} Years")
    
    # 변수 정의: Stocks, Long-term Bonds, Short-term Bonds, Gold
    x = [m.continuous_var(name=f"x{i+1}", lb=0) for i in range(4)]
    
    # 자산 비중 합 제약: 나머지 비중 합은 1 - cash_ratio
    remaining_ratio = 1 - cash_ratio
    m.add_constraint(m.sum(x) == remaining_ratio, "Remaining_Allocation_Sum")
    
    # 무위험 자산 제약
    cash_volatility = m.sum(σ[year_idx][i] * x[i] for i in range(4))
    cash_mdd = m.sum(mdd[year_idx][i] * x[i] for i in range(4))
    
    if y1_val == 1:  # Conservative
        m.add_constraint(cash_volatility >= 0.00000001 * cash_ratio, "Volatility_Stable")
        m.add_constraint(cash_mdd >= 0.0000001 * cash_ratio, "MaxDrawdown_Stable")
    elif y2_val == 1:  # Neutral
        m.add_constraint(cash_volatility >= 0.00000001 * cash_ratio, "Volatility_Neutral")
        m.add_constraint(cash_mdd >= 0.00000001 * cash_ratio, "MaxDrawdown_Neutral")
    elif y3_val == 1:  # Aggressive
        m.add_constraint(cash_volatility  >= 0.000000002 * cash_ratio, "Volatility_Aggressive")
        m.add_constraint(cash_mdd >= 0.00000003 * cash_ratio, "MaxDrawdown_Aggressive")
    
    # 위험 제약
    portfolio_volatility = m.sum(σ[year_idx][i] * x[i] for i in range(4))
    portfolio_mdd = m.sum(mdd[year_idx][i] * x[i] for i in range(4))
    
    if y1_val == 1:  # Conservative
        m.add_constraint(portfolio_volatility <= 0.103 * remaining_ratio, "Volatility_Stable")
        m.add_constraint(portfolio_mdd <= 0.21 * remaining_ratio, "MaxDrawdown_Stable")
    elif y2_val == 1:  # Neutral
        m.add_constraint(portfolio_volatility <= 0.134 * remaining_ratio, "Volatility_Neutral")
        m.add_constraint(portfolio_mdd <= 0.23 * remaining_ratio, "MaxDrawdown_Neutral")
    elif y3_val == 1:  # Aggressive
        m.add_constraint(portfolio_volatility <= 0.144 * remaining_ratio, "Volatility_Aggressive")
        m.add_constraint(portfolio_mdd <= 0.29 * remaining_ratio, "MaxDrawdown_Aggressive")
    
    # 목표 함수: 세후 수익률 최대화
    adj_r = [
        r[year_idx][0] * (1 - div_tax) - transaction_tax,  # Stocks
        r[year_idx][1] * (1 - income_tax),                # Long-term Bonds
        r[year_idx][2] * (1 - income_tax),                # Short-term Bonds
        r[year_idx][3] * (1 - corp_tax)                   # Gold
    ]
    m.set_objective("max", m.sum(adj_r[i] * x[i] for i in range(4)))
    
    solution = m.solve()
    if solution:
        portfolio_volatility_val = portfolio_volatility.solution_value
        portfolio_mdd_val = portfolio_mdd.solution_value
        print(f"[Step 2: Asset Allocation after Risk Hedging]")
        for i, asset_name in enumerate(["Stocks", "Long-term Bonds", "Short-term Bonds", "Gold"]):
            print(f"  - {asset_name}: {solution[x[i]]:.2%}")
        print(f"  - Maximum Expected Return (Objective): {solution.objective_value:.2%}")
        print(f"  - Maximum Allowed Volatility: {portfolio_volatility_val:.2%}")
        print(f"  - Maximum Drawdown (MDD): {portfolio_mdd_val:.2%}")
        print("-" * 50)
    else:
        print(f"[Step 2: Remaining Asset Optimization Failed] Period: {year}years, Investor Type: Conservative={y1_val}, Neutral={y2_val}, Aggressive={y3_val}")
        print("-" * 50)

# 실행
for year in [1, 3, 5]:
    for y1_val in [1, 0]:
        for y2_val in [1, 0]:
            for y3_val in [1, 0]:
                if y1_val + y2_val + y3_val == 1:
                    cash_ratio = optimize_full_portfolio(y1_val, y2_val, y3_val, year)
                    if cash_ratio is not None:
                        optimize_remaining_portfolio(cash_ratio, y1_val, y2_val, y3_val, year)


[Step 1: Cash/Gold Allocation & Risk Hedging] Period: 1years, Investor Type: Conservative=1, Neutral=0, Aggressive=0
  - Cash/Gold allocation is set to 29.93% of the portfolio.

[Step 2: Asset Allocation after Risk Hedging]
  - Stocks: 44.80%
  - Long-term Bonds: 0.00%
  - Short-term Bonds: 25.27%
  - Gold: 0.00%
  - Maximum Expected Return (Objective): 10.77%
  - Maximum Allowed Volatility: 7.22%
  - Maximum Drawdown (MDD): 3.95%
--------------------------------------------------
[Step 1: Cash/Gold Allocation & Risk Hedging] Period: 1years, Investor Type: Conservative=0, Neutral=1, Aggressive=0
  - Cash/Gold allocation is set to 8.84% of the portfolio.

[Step 2: Asset Allocation after Risk Hedging]
  - Stocks: 81.44%
  - Long-term Bonds: 0.00%
  - Short-term Bonds: 9.71%
  - Gold: 0.00%
  - Maximum Expected Return (Objective): 18.30%
  - Maximum Allowed Volatility: 12.21%
  - Maximum Drawdown (MDD): 6.85%
--------------------------------------------------
[Step 1: Cash/Gold Allocation

In [5]:
from docplex.mp.model import Model

# 데이터 정의
risk_free_r = [[0.051], [0.023], [0.019]]  # 현Gold 수익률 (1years, 3years, 5years)
r = [
    [0.262,-0.062, 0.042, 0.127],  # Stocks, Long-term Bonds, Short-term Bonds, Gold (1years)
    [0.1, -0.053, -0.002, 0.023],  # 3years
    [0.156, 0.002,0.012, 0.095]   # 5years
]
σ = [
    [0.147, 0.099, 0.025, 0.151],  # 1years
    [0.176, 0.085, 0.022, 0.142],  # 3years
    [0.185, 0.077, 0.019, 0.145]   # 5years
]
mdd = [
    [0.083, 0.199, 0.009, 0.072],  # 1years
    [0.239, 0.217, 0.054, 0.159],  # 3years
    [0.239, 0.232, 0.054, 0.181]   # 5years
]
year_mapping = {1: 0, 3: 1, 5: 2}

# 세율 정의
income_tax = 0.154
corp_tax = 0.19
div_tax = 0.15
transaction_tax = 0.0023

# 1단계: 전체 최적화 (현Gold 포함)
def optimize_full_portfolio(y1_val, y2_val, y3_val, year):
    year_idx = year_mapping[year]
    m = Model(name=f"Full Portfolio Optimization - {year} Years")
    
    # 변수 정의
    x = [m.continuous_var(name=f"x{i+1}", lb=0) for i in range(4)]  # Stocks, Long-term Bonds, Short-term Bonds, Gold
    c = m.continuous_var(name="cash", lb=0)  # 현Gold

    # 자산 비중 합 제약
    m.add_constraint(m.sum(x) + c == 1, "Asset_Allocation_Sum")
    
    # 위험 제약
    portfolio_volatility = m.sum(σ[year_idx][i] * x[i] for i in range(4))
    portfolio_mdd = m.sum(mdd[year_idx][i] * x[i] for i in range(4))
    
    if y1_val == 1:  # Conservative
        m.add_constraint(portfolio_volatility <= 0.103, "Volatility_Stable")
        m.add_constraint(portfolio_mdd <= 0.21, "MaxDrawdown_Stable")
    elif y2_val == 1:  # Neutral
        m.add_constraint(portfolio_volatility <= 0.134, "Volatility_Neutral")
        m.add_constraint(portfolio_mdd <= 0.23, "MaxDrawdown_Neutral")
    elif y3_val == 1:  # Aggressive
        m.add_constraint(portfolio_volatility <= 0.144, "Volatility_Aggressive")
        m.add_constraint(portfolio_mdd <= 0.29, "MaxDrawdown_Aggressive")
    
    # 목표 함수: 세후 수익률 최대화
    adj_r = [
        r[year_idx][0] * (1 - div_tax) - transaction_tax,  # Stocks
        r[year_idx][1] * (1 - income_tax),                # Long-term Bonds
        r[year_idx][2] * (1 - income_tax),                # Short-term Bonds
        r[year_idx][3] * (1 - corp_tax),                  # Gold
    ]
    adj_risk_free_r= risk_free_r[year_idx][0] * (1 - income_tax)         # 현Gold
    
    m.set_objective("max", m.sum(adj_r[i] * x[i] for i in range(4)) + adj_risk_free_r * c)
    cash_ratio=adj_risk_free_r * c
    
    solution = m.solve()
    if solution:
        cash_ratio = solution[c]
        portfolio_volatility_val = portfolio_volatility.solution_value
        portfolio_mdd_val = portfolio_mdd.solution_value
        print(f"[Step 1: Cash/Gold Allocation & Risk Hedging] Period: {year}years, Investor Type: Conservative={y1_val}, Neutral={y2_val}, Aggressive={y3_val}")
        print(f"  - Cash/Gold allocation is set to {cash_ratio:.2%} of the portfolio.\n")
        return cash_ratio
    else:
        print(f"[Step 1: Cash/Gold Optimization Failed] Period: {year}years, Investor Type: Conservative={y1_val}, Neutral={y2_val}, Aggressive={y3_val}")
        return None

# 2단계: 나머지 자산 최적화
def optimize_remaining_portfolio(cash_ratio, y1_val, y2_val, y3_val, year):
    year_idx = year_mapping[year]
    m = Model(name=f"Remaining Portfolio Optimization - {year} Years")
    
    # 변수 정의: Stocks, Long-term Bonds, Short-term Bonds, Gold
    x = [m.continuous_var(name=f"x{i+1}", lb=0) for i in range(4)]
    
    # 자산 비중 합 제약: 나머지 비중 합은 1 - cash_ratio
    remaining_ratio = 1 - cash_ratio
    m.add_constraint(m.sum(x) == remaining_ratio, "Remaining_Allocation_Sum")
    
    # 무위험 자산 제약
    cash_volatility = m.sum(σ[year_idx][i] * x[i] for i in range(4))
    cash_mdd = m.sum(mdd[year_idx][i] * x[i] for i in range(4))
    
    if y1_val == 1:  # Conservative
        m.add_constraint(cash_volatility >= 0.00000001 * cash_ratio, "Volatility_Stable")
        m.add_constraint(cash_mdd >= 0.0000001 * cash_ratio, "MaxDrawdown_Stable")
    elif y2_val == 1:  # Neutral
        m.add_constraint(cash_volatility >= 0.00000001 * cash_ratio, "Volatility_Neutral")
        m.add_constraint(cash_mdd >= 0.00000001 * cash_ratio, "MaxDrawdown_Neutral")
    elif y3_val == 1:  # Aggressive
        m.add_constraint(cash_volatility  >= 0.000000002 * cash_ratio, "Volatility_Aggressive")
        m.add_constraint(cash_mdd >= 0.00000003 * cash_ratio, "MaxDrawdown_Aggressive")
    
    # 위험 제약
    portfolio_volatility = m.sum(σ[year_idx][i] * x[i] for i in range(4))
    portfolio_mdd = m.sum(mdd[year_idx][i] * x[i] for i in range(4))
    
    if y1_val == 1:  # Conservative
        m.add_constraint(portfolio_volatility <= 0.103 * remaining_ratio, "Volatility_Stable")
        m.add_constraint(portfolio_mdd <= 0.21 * remaining_ratio, "MaxDrawdown_Stable")
    elif y2_val == 1:  # Neutral
        m.add_constraint(portfolio_volatility <= 0.134 * remaining_ratio, "Volatility_Neutral")
        m.add_constraint(portfolio_mdd <= 0.23 * remaining_ratio, "MaxDrawdown_Neutral")
    elif y3_val == 1:  # Aggressive
        m.add_constraint(portfolio_volatility <= 0.144 * remaining_ratio, "Volatility_Aggressive")
        m.add_constraint(portfolio_mdd <= 0.29 * remaining_ratio, "MaxDrawdown_Aggressive")
    
    # 목표 함수: 세후 수익률 최대화
    adj_r = [
        r[year_idx][0] * (1 - div_tax) - transaction_tax,  # Stocks
        r[year_idx][1] * (1 - income_tax),                # Long-term Bonds
        r[year_idx][2] * (1 - income_tax),                # Short-term Bonds
        r[year_idx][3] * (1 - corp_tax)                   # Gold
    ]
    m.set_objective("max", m.sum(adj_r[i] * x[i] for i in range(4)))
    
    solution = m.solve()
    if solution:
        portfolio_volatility_val = portfolio_volatility.solution_value
        portfolio_mdd_val = portfolio_mdd.solution_value
        print(f"[Step 2: Asset Allocation after Risk Hedging]")
        for i, asset_name in enumerate(["Stocks", "Long-term Bonds", "Short-term Bonds", "Gold"]):
            print(f"  - {asset_name}: {solution[x[i]]:.2%}")
        print(f"  - Maximum Expected Return (Objective): {solution.objective_value:.2%}")
        print(f"  - Maximum Allowed Volatility: {portfolio_volatility_val:.2%}")
        print(f"  - Maximum Drawdown (MDD): {portfolio_mdd_val:.2%}")
        print("-" * 50)
    else:
        print(f"[Step 2: Remaining Asset Optimization Failed] Period: {year}years, Investor Type: Conservative={y1_val}, Neutral={y2_val}, Aggressive={y3_val}")
        print("-" * 50)

# 실행
for year in [1, 3, 5]:
    for y1_val in [1, 0]:
        for y2_val in [1, 0]:
            for y3_val in [1, 0]:
                if y1_val + y2_val + y3_val == 1:
                    cash_ratio = optimize_full_portfolio(y1_val, y2_val, y3_val, year)
                    if cash_ratio is not None:
                        optimize_remaining_portfolio(cash_ratio, y1_val, y2_val, y3_val, year)


[Step 1: Cash/Gold Allocation & Risk Hedging] Period: 1years, Investor Type: Conservative=1, Neutral=0, Aggressive=0
  - Cash/Gold allocation is set to 29.93% of the portfolio.

[Step 2: Asset Allocation after Risk Hedging]
  - Stocks: 44.80%
  - Long-term Bonds: 0.00%
  - Short-term Bonds: 25.27%
  - Gold: 0.00%
  - Maximum Expected Return (Objective): 10.77%
  - Maximum Allowed Volatility: 7.22%
  - Maximum Drawdown (MDD): 3.95%
--------------------------------------------------
[Step 1: Cash/Gold Allocation & Risk Hedging] Period: 1years, Investor Type: Conservative=0, Neutral=1, Aggressive=0
  - Cash/Gold allocation is set to 8.84% of the portfolio.

[Step 2: Asset Allocation after Risk Hedging]
  - Stocks: 81.44%
  - Long-term Bonds: 0.00%
  - Short-term Bonds: 9.71%
  - Gold: 0.00%
  - Maximum Expected Return (Objective): 18.30%
  - Maximum Allowed Volatility: 12.21%
  - Maximum Drawdown (MDD): 6.85%
--------------------------------------------------
[Step 1: Cash/Gold Allocation